In [1]:
import os, sys
import numpy as np
import pandas as pd
import shap
import matplotlib.pyplot as plt
import joblib

# paths
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.insert(0, PROJECT_ROOT)

from src.io_utils import load_df

X_TEST_PATH   = os.path.join(PROJECT_ROOT, "data/processed/X_test.csv")
Y_TEST_PATH   = os.path.join(PROJECT_ROOT, "data/processed/y_test.csv")
X_TRAIN_PATH  = os.path.join(PROJECT_ROOT, "data/processed/X_train.csv")

RF_MODEL_PATH  = os.path.join(PROJECT_ROOT, "models/rf_baseline.pkl")
XGB_MODEL_PATH = os.path.join(PROJECT_ROOT, "models/xgb_classifier.pkl")

FIG_DIR     = os.path.join(PROJECT_ROOT, "figures")
REPORTS_DIR = os.path.join(PROJECT_ROOT, "reports")
os.makedirs(FIG_DIR, exist_ok=True)
os.makedirs(REPORTS_DIR, exist_ok=True)

# data
X_test = load_df(X_TEST_PATH)
y_test = load_df(Y_TEST_PATH)["Target"]
X_train_bg = load_df(X_TRAIN_PATH)

# models
rf_model  = joblib.load(RF_MODEL_PATH)
xgb_model = joblib.load(XGB_MODEL_PATH)

print("Loaded test shape:", X_test.shape)


/home/glinux/Projects/Skole/AnvendtData/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded test shape: (885, 13)


In [2]:
# Pick one; you can run the notebook twice to compare both
model = rf_model        # or: xgb_model
model_name = "XGBoost"  # or: "XGBoost"

In [3]:
# background sample for interventional SHAP (future-proof)
bg_size = min(500, len(X_train_bg))
background = X_train_bg.sample(n=bg_size, random_state=42)

# sample test rows to explain
X_sample = X_test.sample(n=min(500, len(X_test)), random_state=42)

backend = None
try:
    if model_name == "Random Forest":
        explainer = shap.TreeExplainer(
            model,
            data=background,
            feature_perturbation="interventional",
            model_output="raw"
        )
        shap_values = explainer.shap_values(X_sample)
        backend = "tree-interventional"

    elif model_name == "XGBoost":
        explainer = shap.TreeExplainer(
            model.get_booster(),
            data=background,
            feature_perturbation="interventional"
        )
        shap_values = explainer.shap_values(X_sample)
        backend = "xgb-tree-interventional"

    else:
        raise ValueError("Unknown model type")

except Exception as e:
    print("TreeExplainer failed, falling back to model-agnostic SHAP:", e)
    explainer = shap.Explainer(model.predict_proba, background, algorithm="permutation")
    shap_values = explainer(X_sample)
    backend = "model-agnostic"

print(f"SHAP backend used: {backend}")


TreeExplainer failed, falling back to model-agnostic SHAP: 'RandomForestClassifier' object has no attribute 'get_booster'


PermutationExplainer explainer: 501it [17:58,  2.16s/it]                         

SHAP backend used: model-agnostic


In [4]:
def _drop_bias_col(arr, n_feat):
    return arr[:, :-1] if arr.shape[1] == n_feat + 1 else arr

def _fix_shapes(shap_vals, X):
    # list of arrays (multiclass)
    if isinstance(shap_vals, list):
        return [_drop_bias_col(sv, X.shape[1]) for sv in shap_vals]
    # Explanation object (new API)
    try:
        from shap._explanation import Explanation  # noqa: F401
        if isinstance(shap_vals, Explanation):
            return shap_vals
    except Exception:
        pass
    # ndarray (legacy path)
    if isinstance(shap_vals, np.ndarray):
        return _drop_bias_col(shap_vals, X.shape[1])
    return shap_vals

fixed = _fix_shapes(shap_values, X_sample)


In [5]:
fig_base = f"{model_name.lower()}"

# Case A: list of arrays (multiclass tree backend)
if isinstance(fixed, list):
    shap.summary_plot(fixed, X_sample, show=False, plot_size=(8,6))
    plt.tight_layout()
    plt.savefig(os.path.join(FIG_DIR, f"{fig_base}_shap_summary.png"), dpi=150, bbox_inches="tight")
    plt.close()

    # Per-class summaries (optional)
    labels = ["Dropout", "Graduate", "Enrolled"]
    for i, lbl in enumerate(labels[:len(fixed)]):
        shap.summary_plot(fixed[i], X_sample, show=False, plot_size=(8,6))
        plt.title(f"{model_name} — SHAP Summary ({lbl})")
        plt.tight_layout()
        plt.savefig(os.path.join(FIG_DIR, f"{fig_base}_shap_{lbl.lower()}.png"), dpi=150, bbox_inches="tight")
        plt.close()

else:
    # Case B: Explanation object (model-agnostic)
    try:
        from shap._explanation import Explanation
        if isinstance(fixed, Explanation):
            shap.plots.beeswarm(fixed, max_display=20, show=False)
            plt.title(f"{model_name} — SHAP Summary (Beeswarm)")
            plt.tight_layout()
            plt.savefig(os.path.join(FIG_DIR, f"{fig_base}_shap_summary.png"), dpi=150, bbox_inches="tight")
            plt.close()
        else:
            # Case C: ndarray -> legacy summary_plot
            shap.summary_plot(fixed, X_sample, show=False, plot_size=(8,6))
            plt.tight_layout()
            plt.savefig(os.path.join(FIG_DIR, f"{fig_base}_shap_summary.png"), dpi=150, bbox_inches="tight")
            plt.close()
    except Exception:
        shap.summary_plot(fixed, X_sample, show=False, plot_size=(8,6))
        plt.tight_layout()
        plt.savefig(os.path.join(FIG_DIR, f"{fig_base}_shap_summary.png"), dpi=150, bbox_inches="tight")
        plt.close()


/tmp/ipykernel_2335/4130512506.py:36: FutureWarning: The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence this warning.
  shap.summary_plot(fixed, X_sample, show=False, plot_size=(8,6))


In [6]:
# --- Robust global mean |SHAP| (always returns 1-D per feature) ---
def global_mean_abs_shap(fixed_values, X):
    import numpy as np
    # Case 1: list of per-class arrays
    if isinstance(fixed_values, list):
        # average mean|SHAP| across classes
        per_class = [np.abs(sv).mean(axis=0) for sv in fixed_values]  # (n_features,)
        return np.mean(per_class, axis=0)  # (n_features,)

    # Try Explanation API
    try:
        from shap._explanation import Explanation  # type: ignore
        if isinstance(fixed_values, Explanation):
            vals = fixed_values.values
            # shapes: (n_samples, n_features) OR (n_samples, n_features, n_outputs)
            if vals.ndim == 2:
                return np.abs(vals).mean(axis=0)  # (n_features,)
            elif vals.ndim == 3:
                return np.abs(vals).mean(axis=(0, 2))  # (n_features,)
    except Exception:
        pass

    # ndarray fallback
    arr = np.asarray(fixed_values)
    if arr.ndim == 2 and arr.shape[1] in (X.shape[1], X.shape[1] + 1):
        # (n_samples, n_features [+ bias])
        if arr.shape[1] == X.shape[1] + 1:
            arr = arr[:, :-1]
        return np.abs(arr).mean(axis=0)
    elif arr.ndim == 3:
        # could be (n_outputs, n_samples, n_features) or (n_samples, n_features, n_outputs)
        if arr.shape[-1] == X.shape[1]:   # (..., n_features)
            return np.abs(arr).mean(axis=(0, 1))
        elif arr.shape[-2] == X.shape[1]: # (..., n_features, ...)
            return np.abs(arr).mean(axis=(0, 2))
    raise ValueError(f"Unhandled SHAP value shape: {arr.shape}")

mean_abs = global_mean_abs_shap(fixed, X_sample)

# now this will be 1-D and align with columns
imp_df = (
    pd.DataFrame({"feature": X_sample.columns, "mean_abs_shap": mean_abs})
      .sort_values("mean_abs_shap", ascending=False)
)

imp_path = os.path.join(REPORTS_DIR, f"{fig_base}_shap_importance.csv")
imp_df.to_csv(imp_path, index=False)
print("Saved:", imp_path)
imp_df.head(10)


Saved: /home/glinux/Projects/Skole/AnvendtData/reports/xgboost_shap_importance.csv


,feature,mean_abs_shap
2,overall_approval_rate,0.126282
1,total_approved_units,0.053210
0,average_grade,0.047040
3,Tuition fees up to date,0.033748
6,Age at enrollment,0.031198
5,Scholarship holder,0.023923
7,Gender,0.009709
9,Unemployment rate,0.009395
11,GDP,0.008966
10,Inflation rate,0.008132


In [7]:
labels = ["Dropout", "Graduate", "Enrolled"]

if isinstance(fixed, list) and len(fixed) == 3:
    per_class_tables = []
    for i, lbl in enumerate(labels):
        mean_abs_i = np.abs(fixed[i]).mean(0)
        df_i = pd.DataFrame({
            "feature": X_sample.columns,
            "mean_abs_shap": mean_abs_i,
            "class": lbl
        }).sort_values("mean_abs_shap", ascending=False).head(10)
        per_class_tables.append(df_i)
    per_class_df = pd.concat(per_class_tables, ignore_index=True)
    out_path = os.path.join(REPORTS_DIR, f"{fig_base}_shap_top10_per_class.csv")
    per_class_df.to_csv(out_path, index=False)
    print("Saved per-class top10:", out_path)
else:
    print("Per-class table skipped (not a multiclass tree backend).")


Per-class table skipped (not a multiclass tree backend).


In [8]:
# ---- Robust dependence plots (manual scatter) ----
from shap._explanation import Explanation  # noqa: F401

top_feats = imp_df["feature"].head(5).tolist()
labels = ["Dropout", "Graduate", "Enrolled"]

def _dep_plot_manual(feat, shap_vals, X, save_path, class_idx=0):
    j = X.columns.get_loc(feat)
    x = X.iloc[:, j].values

    # Case A: list of per-class arrays (TreeExplainer, multiclass)
    if isinstance(shap_vals, list):
        sv = shap_vals[class_idx]  # (n_samples, n_features)
        y = sv[:, j]

    # Case B: Explanation object (model-agnostic/new API)
    elif isinstance(shap_vals, Explanation):
        vals = shap_vals.values
        # shapes: (n_samples, n_features) OR (n_samples, n_features, n_outputs)
        if vals.ndim == 2:
            y = vals[:, j]
        elif vals.ndim == 3:
            # pick the class dimension
            y = vals[:, j, class_idx]
        else:
            raise ValueError(f"Unexpected Explanation.values ndim: {vals.ndim}")

    # Case C: plain ndarray (legacy)
    else:
        arr = np.asarray(shap_vals)
        if arr.ndim == 2:                  # (n_samples, n_features [+1 bias])
            if arr.shape[1] == X.shape[1] + 1:
                arr = arr[:, :-1]
            y = arr[:, j]
        elif arr.ndim == 3:
            # try (n_outputs, n_samples, n_features)
            if arr.shape[0] in (2, 3) and arr.shape[2] == X.shape[1]:
                y = arr[class_idx, :, j]
            # or (n_samples, n_features, n_outputs)
            elif arr.shape[2] in (2, 3) and arr.shape[1] == X.shape[1]:
                y = arr[:, j, class_idx]
            else:
                raise ValueError(f"Unhandled ndarray shape for dependence: {arr.shape}")
        else:
            raise ValueError(f"Unhandled ndarray ndim: {arr.ndim}")

    # Clean NaNs / inf and ensure same length
    mask = np.isfinite(x) & np.isfinite(y)
    x, y = x[mask], y[mask]
    if x.shape[0] != y.shape[0]:
        raise ValueError(f"x and y must be same size; got {x.shape[0]} and {y.shape[0]}")

    # Plot
    plt.figure(figsize=(6, 4))
    plt.scatter(x, y, s=12, alpha=0.6)
    plt.xlabel(feat)
    plt.ylabel(f"SHAP value ({labels[class_idx]})")
    plt.title(f"{model_name} — Dependence: {feat} → {labels[class_idx]}")
    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches="tight")
    plt.close()

# Generate plots for top 5 features, focused on Dropout class by default (class_idx=0)
for feat in top_feats:
    out = os.path.join(FIG_DIR, f"{fig_base}_shap_dependence_{feat}.png")
    _dep_plot_manual(feat, fixed, X_sample, out, class_idx=0)

print("Saved dependence plots for:", ", ".join(top_feats))


Saved dependence plots for: overall_approval_rate, total_approved_units, average_grade, Tuition fees up to date, Age at enrollment
